## Table List in Dataset

<img src="images/field_list_sample_snowflake.png">

## Import Library

In [ ]:
# Import Library
import pandas as pd
import os

pd.options.mode.chained_assignment = None
pd.options.display.float_format = '{:.2f}'.format

## Read Data

In [ ]:
# Read raw data from csv
data_dir = "data"# Add path to folder contained dataset file
customer_df = pd.read_csv(f"{data_dir}/CUSTOMER.csv") 
lineitem_df = pd.read_csv(f"{data_dir}/LINEITEM.csv")
nation_df = pd.read_csv(f"{data_dir}/NATION.csv")
order_df = pd.read_csv(f"{data_dir}/ORDER.csv")
part_df = pd.read_csv(f"{data_dir}/PART.csv")
partsupp_df = pd.read_csv(f"{data_dir}/PARTSUPP.csv")
region_df = pd.read_csv(f"{data_dir}/REGION.csv")
supplier_df = pd.read_csv(f"{data_dir}/SUPPLIER.csv")

## Data Exploration

#### Lineitem

`DataFrame.info()`
> A method to prints a summary of the DataFrame including:
> - Indexes
> - Columns
> - Non-null values
> - Data types 
> - Memory usage

In [ ]:
# Display a summary info of Lineitem.
lineitem_df.info()

`DataFrame.describe()`
> A method to display descriptive statistics (such as `count`, `mean`, `std`, `min`, `max`, `percentiles` for numeric data) of the DataFrame.

`DataFrame.apply(func, axis=0)` 
> A method to apply a function along an axis (either `axis=0`: DataFrame’s index (default) or `axis=1`: the DataFrame’s columns) of the DataFrame.

In [ ]:
# Display the descriptive statistics of Lineitem
lineitem_df.describe()

`DataFrame.head(n=5)`
> A method to return the first n rows (default=5) for the object based on position.

In [ ]:
# Display sample of Lineitem.
lineitem_df.head()

#### Order

In [ ]:
# Display a summary info of Order.
order_df.info()

In [ ]:
# Display the descriptive statistics of Order
order_df.describe()

In [ ]:
# Display sample of Order
order_df.head()

#### Customer & Supplier

In [ ]:
# Explore data for other table
print("Customer")
customer_df.info()

print("\nSupplier")
supplier_df.info()

In [ ]:
customer_df.describe()

In [ ]:
supplier_df.describe()

In [ ]:
customer_df.head()

In [ ]:
supplier_df.head()

#### Entity Relationship

<img src="images/dataset_entity_relationship.png">

## Data Cleansing

#### Business logic for LineItem table's field
- Shouldn't be any duplicate row (reference by value in all column) >> Move duplicate column cleansing activity to top
- L_QUANTITY should have value more than 0, not be decimal number, not be negative value and should not contain NaN value
- L_LINENUMBER should have value more than 0, not be decimal number, not be negative value and not contain NaN value
- L_LINESTATUS should contain only "F", "O" status and should not contain NaN value >> *To be check, didn't cleaned in this lab*
- L_SHIPMODE should not contain NaN value and only value list below allowed >> *To be check, didn't cleaned in this lab*
    - TRUCK
    - AIR
    - MAIL
    - SHIP
    - RAIL
    - FOB
    - REG AIR

#### Lineitem

In [ ]:
# Show duplicate record
lineitem_df[lineitem_df.duplicated(keep=False)].sort_values(lineitem_df.columns.tolist())

In [ ]:
# Remove duplicate records
lineitem_df = lineitem_df.drop_duplicates()

In [ ]:
# Drop records that contains nan
lineitem_df = lineitem_df.dropna(subset=['L_LINENUMBER','L_QUANTITY','L_LINESTATUS','L_SHIPMODE'])

# There are some other method to deal with na by filling the records with other value
#lineitem_df['L_LINENUMBER'] = lineitem_df['L_LINENUMBER'].ffill()
#lineitem_df['L_QUANTITY'] = lineitem_df['L_QUANTITY'].bfill()
#lineitem_df['L_LINESTATUS'] = lineitem_df['L_LINESTATUS'].fillna(lineitem_df['L_LINESTATUS'].mode())
#lineitem_df['L_SHIPMODE'] = lineitem_df['L_SHIPMODE'].fillna(lineitem_df['L_SHIPMODE'].mode())

In [ ]:
# Convert decimal to integer (cut decimal) and inverse negative
lineitem_df['L_LINENUMBER'] = lineitem_df['L_LINENUMBER'].astype(int) 
lineitem_df['L_LINENUMBER'] = lineitem_df['L_LINENUMBER'].abs() # For more clarification ask business for clarification - abs, outlier, drop

In [ ]:
# LineNumber should not contain 0
lineitem_df = lineitem_df[lineitem_df['L_LINENUMBER']!=0].copy()

In [ ]:
# Convert decimal to integer (cut decimal) and inverse negative
lineitem_df['L_QUANTITY'] = lineitem_df['L_QUANTITY'].astype(int)
lineitem_df['L_QUANTITY'] = lineitem_df['L_QUANTITY'].abs() # For more clarification ask business for clarification - abs, outlier, drop

In [ ]:
# Quantity should have value more than 0
lineitem_df = lineitem_df[lineitem_df['L_QUANTITY']!=0].copy()

In [ ]:
# Show number of record group by each value
lineitem_df['L_LINESTATUS'].value_counts()

In [ ]:
# Replace out-of-scope row with correct value for LINESTATUS column
lineitem_df.loc[(lineitem_df['L_LINESTATUS'].str.contains('F')) & ~(lineitem_df['L_LINESTATUS'].str.contains('O')), 'L_LINESTATUS'] = 'F'
lineitem_df.loc[(lineitem_df['L_LINESTATUS'].str.contains('O')) & ~(lineitem_df['L_LINESTATUS'].str.contains('F')), 'L_LINESTATUS'] = 'O'

In [ ]:
# Remove any leftover out-of-scope value for LINESTATUS column
lineitem_df = lineitem_df[lineitem_df['L_LINESTATUS'].isin(['F','O'])]

In [ ]:
# Show number of record group by each value
lineitem_df['L_SHIPMODE'].value_counts()

In [ ]:
# Remove out-of-scope row for LINESTATUS column
lineitem_df = lineitem_df[lineitem_df['L_SHIPMODE'].isin(['TRUCK', 'AIR', 'MAIL', 'SHIP', 'RAIL', 'FOB', 'REG AIR'])]

#### Business logic for Order table's field
- Shouldn't be any duplicate row (reference by value in all column)
- O_ORDERSTATUS should contain only "F", "O" and "P" status and not contain NaN value
- O_ORDERDATE valid order year should in between 1992-1998 and should not contain NaN value
- O_ORDERPRIORITY should not contain NaN value and only value list below allowed
    
    - 1-URGENT
    - 2-HIGH
    - 3-MEDIUM
    - 4-LOW
    - 5-NOT SPECIFIED 


#### Order

In [ ]:
# Show duplicate records
order_df[order_df.duplicated(keep=False)].sort_values(order_df.columns.tolist())

In [ ]:
# Remove duplicate records
order_df = order_df.drop_duplicates()

In [ ]:
# Drop nan records
order_df = order_df.dropna(subset=['O_ORDERSTATUS', 'O_ORDERPRIORITY','O_ORDERDATE'])

In [ ]:
# Show number of record group by each value
order_df['O_ORDERPRIORITY'].value_counts()

In [ ]:
# Fix records with typo
order_df.loc[order_df['O_ORDERPRIORITY'].str.contains('1'),'O_ORDERPRIORITY'] = '1-URGENT'
order_df.loc[order_df['O_ORDERPRIORITY'].str.contains('2'),'O_ORDERPRIORITY'] = '2-HIGH'
order_df.loc[order_df['O_ORDERPRIORITY'].str.contains('3'),'O_ORDERPRIORITY'] = '3-MEDIUM'
order_df.loc[order_df['O_ORDERPRIORITY'].str.contains('4'),'O_ORDERPRIORITY'] = '4-LOW'
order_df.loc[order_df['O_ORDERPRIORITY'].str.contains('5'),'O_ORDERPRIORITY'] = '5-NOT SPECIFIED'

In [ ]:
# Remove any leftover records with typo
order_df = order_df[order_df['O_ORDERPRIORITY'].isin(['1-URGENT','2-HIGH','3-MEDIUM','4-LOW','5-NOT SPECIFIED'])]

In [ ]:
# Show number of record group by each value
order_df['O_ORDERSTATUS'].value_counts()

In [ ]:
# Fix records with typo
order_df.loc[(order_df['O_ORDERSTATUS'].str.contains('F')) & ~(order_df['O_ORDERSTATUS'].str.contains('O|P')), 'O_ORDERSTATUS'] = 'F'
order_df.loc[(order_df['O_ORDERSTATUS'].str.contains('O')) & ~(order_df['O_ORDERSTATUS'].str.contains('F|P')), 'O_ORDERSTATUS'] = 'O'
order_df.loc[(order_df['O_ORDERSTATUS'].str.contains('P')) & ~(order_df['O_ORDERSTATUS'].str.contains('F|O')), 'O_ORDERSTATUS'] = 'P'

In [ ]:
# Remove any leftover records with typo
order_df = order_df[order_df['O_ORDERSTATUS'].isin(['F','O','P'])]

In [ ]:
# Extract day, month and year from ORDERDATE (datetime format)
order_df['O_ORDERDATE'] = pd.to_datetime(order_df['O_ORDERDATE'])

order_df['O_ORDERYEAR'] = order_df['O_ORDERDATE'].dt.year
order_df['O_ORDERDAY'] = order_df['O_ORDERDATE'].dt.day
order_df['O_ORDERMONTH'] = order_df['O_ORDERDATE'].dt.month

In [ ]:
# Show number of record group by year
order_df['O_ORDERYEAR'].value_counts()

In [ ]:
# Remove invalid order year
order_df = order_df[order_df['O_ORDERYEAR'].between(1992, 1998)]

#order_df = order_df[order_df['O_ORDERYEAR'].ge(1992) & order_df['O_ORDERYEAR'].le(1998)]
#order_df = order_df[(order_df['O_ORDERYEAR'] >= 1992) & (order_df['O_ORDERYEAR'] <= 1998)]

## Data Validation

#### LineItem

In [ ]:
# Show dataframe information
lineitem_df.info()

In [ ]:
# Check duplicate row
lineitem_duplicate_check = lineitem_df.duplicated().sum()

print("LineItem dataset contain " + str(lineitem_duplicate_check) + " duplicate rows")

In [ ]:
# Check NaN value in specific column
quantity_nan_check = lineitem_df['L_QUANTITY'].isnull().sum()
linenumber_nan_check = lineitem_df['L_LINENUMBER'].isnull().sum()
linestatus_nan_check = lineitem_df['L_LINESTATUS'].isnull().sum()
shipmode_nan_check = lineitem_df['L_SHIPMODE'].isnull().sum()

print("L_QUANTITY have " + str(quantity_nan_check) + " NaN row")
print("L_LINENUMBER have " + str(linenumber_nan_check) + " NaN row")
print("L_LINESTATUS have " + str(linestatus_nan_check) + " NaN row")
print("L_SHIPMODE have " + str(shipmode_nan_check) + " NaN row")

In [ ]:
# Check 0 or negative value in specific column
quantity_negativeorzero_check = len(lineitem_df[lineitem_df['L_QUANTITY']<=0])
linenumber_negativeorzero_check = len(lineitem_df[lineitem_df['L_LINENUMBER']<=0])

print("L_QUANTITY contain " + str(quantity_negativeorzero_check) + " rows zero or negative value")
print("L_LINENUMBER contain " + str(linenumber_negativeorzero_check) + " rows zero or negative value")

In [ ]:
# Check out-of-scope value in specific
linestatus_outofscope_check = len(lineitem_df[~lineitem_df['L_LINESTATUS'].isin(['F', 'O'])])
shipmode_outofscope_check = len(lineitem_df[~lineitem_df['L_SHIPMODE'].isin(['TRUCK', 'AIR', 'MAIL', 'SHIP', 'RAIL', 'FOB', 'REG AIR'])])

print("L_LINESTATUS contain " + str(linestatus_outofscope_check) + " rows out-of-scope value")
print("L_SHIPMODE contain " + str(shipmode_outofscope_check) + " rows out-of-scope value")

#### Order

In [ ]:
# Show dataframe information
order_df.info()

In [ ]:
# Check duplicate row
order_duplicate_check = order_df.duplicated().sum()

print(f"LineItem dataset contain {str(order_duplicate_check)} duplicate rows")

In [ ]:
# Check NaN value in specific column
orderstatus_nan_check = order_df['O_ORDERSTATUS'].isnull().sum()
orderdate_nan_check = order_df['O_ORDERDATE'].isnull().sum()
orderpriority_nan_check = order_df['O_ORDERPRIORITY'].isnull().sum()

print("O_ORDERSTATUS have " + str(orderstatus_nan_check) + " NaN row")
print("O_ORDERDATE have " + str(orderdate_nan_check) + " NaN row")
print("O_ORDERPRIORITY have " + str(orderpriority_nan_check) + " NaN row")

In [ ]:
# Check out-of-scope value in specific column
orderstatus_outofscope_check = len(order_df[~order_df['O_ORDERSTATUS'].isin(['F', 'O', 'P'])])
orderpriotity_outofscope_check = len(order_df[~order_df['O_ORDERPRIORITY'].isin(['1-URGENT', '2-HIGH', '3-MEDIUM', '4-LOW', '5-NOT SPECIFIED'])])

print("O_ORDERSTATUS contain " + str(orderstatus_outofscope_check) + " rows out-of-scope value")
print("O_ORDERPRIORITY contain " + str(orderpriotity_outofscope_check) + " rows out-of-scope value")

In [ ]:
# Check valid year for date column
orderdate_valid_year_check = len(order_df[~order_df['O_ORDERYEAR'].between(1992,1998)])

print("O_ORDERDATE contain " + str(orderdate_valid_year_check) + " invalid year (not in 1992-1998)")

## Data Manipulation

### Denomalization

<img src="images/data_manipulation_denormalization.png">

- Order < Customer < Nation < Region
- Supplier < Nation < Region
- LineItem
- PartSupp
- Part

### Join Data

#### Join Nation with Region

In [ ]:
NR = pd.merge(left=nation_df,
               right=region_df,
               left_on='N_REGIONKEY',
               right_on='R_REGIONKEY',
               how='left')

NR = NR[['N_NATIONKEY','N_NAME','R_NAME']]
NR = NR.rename(columns={"N_NAME": "N_NATION", 
                          "R_NAME": "N_REGION"})

In [ ]:
NR.info()

In [ ]:
NR.head()

In [ ]:
# Compare data between header table and joined dataframe
print("Number of row in header table (Nation) before join : " + str(len(nation_df)))
print("Number of row after joined : " + str(len(NR)))


#### Join Customer with Nation and Region

In [ ]:
CNR = pd.merge(left=customer_df,
                 right=NR,
                 left_on='C_NATIONKEY',
                 right_on='N_NATIONKEY',
                 how='left')

CNR = CNR.drop(['N_NATIONKEY','C_COMMENT'],axis=1)
CNR = CNR.rename(columns={"N_NATION": "C_NATION",
                              "N_REGION": "C_REGION"})

In [ ]:
CNR.info()

In [ ]:
CNR.head()

In [ ]:
# Compare data between header table and joined dataframe
print("Number of row in header table (Customer) before join : " + str(len(customer_df)))
print("Number of row after joined : " + str(len(CNR)))

#### Join Order with Customer, Nation and Region

In [ ]:
OCNR = pd.merge(left=order_df,
                right=CNR,
                left_on="O_CUSTKEY",
                right_on="C_CUSTKEY",
                how='left')

OCNR = OCNR.drop(['C_CUSTKEY'],axis=1)

In [ ]:
OCNR.info()

In [ ]:
OCNR.head()

In [ ]:
# Compare data between header table and joined dataframe
print("Number of row in header table (Order) before join : " + str(len(order_df)))
print("Number of row after joined : " + str(len(OCNR)))

#### Join Supplier with Nation and Region

In [ ]:
SNR = pd.merge(left=supplier_df,
                         right=NR,
                         left_on='S_NATIONKEY',
                         right_on='N_NATIONKEY',
                         how='left')

#SNR = SNR[['S_SUPPKEY','S_NAME','S_ACCTBAL','NATION','REGION']]
SNR = SNR.drop(['N_NATIONKEY'],axis=1)
SNR = SNR.rename(columns={"N_NATION": "S_NATION",
                              "N_REGION": "S_REGION"})

In [ ]:
SNR.info()

In [ ]:
SNR.head()

In [ ]:
# Compare data between header table and joined dataframe
print("Number of row in header table (Supplier) before join : " + str(len(supplier_df)))
print("Number of row after joined : " + str(len(SNR)))

### Feature Creation

In [ ]:
# Calculate lead day
OCNR['LEADDAY'] = OCNR.sort_values(['C_NAME','O_ORDERDATE']).groupby(['C_NAME'],sort=False)['O_ORDERDATE'].diff()

In [ ]:
# Extract day from lead day column
OCNR['LEADDAY'] = OCNR['LEADDAY'].dt.days

In [ ]:
OCNR = OCNR.dropna(subset=['LEADDAY'])

In [ ]:
OCNR['LEADDAY'].value_counts()

## Analytics Product development

### Download Data for visualization in next lab

In [ ]:
#Create output folder if not exist
if not os.path.exists("clean_data"):
    os.makedirs("clean_data")

data_dir = "clean_data"

#Download modeled data into clean data folder
OCNR.to_csv(f"{data_dir}/OCNR.csv")
SNR.to_csv(f"{data_dir}/SNR.csv")
lineitem_df.to_csv(f"{data_dir}/L.csv")
part_df.to_csv(f"{data_dir}/P.csv")
partsupp_df.to_csv(f"{data_dir}/PS.csv")